### Notebook for the label transfer of PBMCs using `scANVI`

- **Developed by:** Carlos Talavera-López Ph.D
- **Würzburg Institute for Systems Immunology & Julius-Maximilian-Universität Würzburg**
- v230809

### Import required modules

In [ ]:
import scvi
import anndata
import warnings
import numpy as np
import scanpy as sc
import pandas as pd
import matplotlib.pyplot as plt

### Set up working environment

In [ ]:
sc.settings.verbosity = 3
sc.logging.print_versions()
sc.settings.set_figure_params(dpi = 180, color_map = 'magma_r', dpi_save = 300, vector_friendly = True, format = 'svg')

In [ ]:
warnings.simplefilter(action = 'ignore')
scvi.settings.seed = 1712
%config InlineBackend.print_figure_kwargs = {'facecolor' : "w"}
%config InlineBackend.figure_format = 'retina'

In [ ]:
arches_params = dict(
    use_layer_norm = "both",
    use_batch_norm = "none",
    encode_covariates = True,
    dropout_rate = 0.2,
    n_layers = 2,
)

### Read in Healthy data

In [ ]:
healthy_raw = sc.read_h5ad('../data/merged_pbmc_yoshida-imyoo_QCed_pre-processed_ctl230810.h5ad')
healthy_raw

### Perform joint QC analysis

#### Select a subset of cells that are unlabeled. 

In [ ]:
healthy_raw.obs['seed_labels'].cat.categories

In [ ]:
healthy_raw.obs['sample'].value_counts()

In [ ]:
if pd.api.types.is_categorical_dtype(healthy_raw.obs['seed_labels']):
    healthy_raw.obs['seed_labels'] = healthy_raw.obs['seed_labels'].astype(str)

healthy_raw.obs['seed_labels'].replace('nan', 'Unknown', inplace = True)
healthy_raw.obs['seed_labels'] = healthy_raw.obs['seed_labels'].astype('category')

In [ ]:
healthy_raw.obs['seed_labels'].cat.categories

### Select highly variable genes

In [ ]:
adata_raw = healthy_raw.copy()
adata = healthy_raw.copy()
adata.layers['counts'] = adata.X.copy()

sc.pp.highly_variable_genes(
    adata,
    flavor = "seurat_v3",
    n_top_genes = 7000,
    layer = "counts",
    batch_key = "samples",
    subset = True
)
adata

### Transfer of annotation with scANVI

In [ ]:
scvi.model.SCVI.setup_anndata(adata, 
                              batch_key = 'donor', 
                              labels_key = "seed_labels",
                              categorical_covariate_keys = ['donor', 'sample', 'generator'],
                              continuous_covariate_keys=['n_genes', 'percent_mito', 'n_counts']
                              )

In [ ]:
scvi_model = scvi.model.SCVI(adata, 
                             n_latent = 50, 
                             n_layers = 3, 
                             dispersion = 'gene-batch', 
                             gene_likelihood = 'nb')

In [ ]:
scvi_model.train()

### Label transfer with `scANVI` 

In [ ]:
scanvi_model = scvi.model.SCANVI.from_scvi_model(scvi_model, 'Unknown')

In [ ]:
scanvi_model.train()

In [ ]:
adata.obs["C_scANVI"] = scanvi_model.predict(adata)

- Extract latent representation

In [ ]:
adata.obsm["X_scANVI"] = scanvi_model.get_latent_representation(adata)

- Visualise corrected dataset

In [ ]:
sc.pp.neighbors(adata, use_rep = "X_scANVI", n_neighbors = 50, metric = 'minkowski')
sc.tl.umap(adata, min_dist = 0.3, spread = 4, random_state = 1712)
sc.pl.umap(adata, frameon = False, color = ['donor', 'cell_states', 'sample', 'generator', 'seed_labels'], size = 1, legend_fontsize = 5, ncols = 3)

### Export annotated object

In [ ]:
SCC0120_1_pbmc_annotated = adata[adata.obs['group'].isin(['SCC0120_1_pbmc'])]
SCC0120_1_pbmc_annotated

In [ ]:
SCC0120_1_pbmc_annotated.obs = SCC0120_1_pbmc_annotated.obs[['orig.ident', 'Age_group', 'BMI', 'COVID_severity', 'COVID_status', 'Ethnicity', 'Group', 'Sex', 'annotation_broad', 'annotation_detailed', 'sample_id', 'seed_labels', 'donor', 'cell_states', 'nCount_RNA', 'nFeature_RNA', 'nCount_HTO', 'nFeature_HTO', 'nCount_CITE', 'nFeature_CITE', 'nCount_PROT', 'nFeature_PROT', 'percent.mt', 'sample', 'HTO_maxID', 'HTO_secondID', 'HTO_margin', 'HTO_classification', 'HTO_classification.global', 'hash.ID', 'tissue', 'condition', 'n_genes', 'doublet_scores', 'hashtag', 'unique', 'group', 'batch', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'pct_counts_ribo', 'percent_mt2', 'n_counts', 'percent_chrY', 'S_score', 'G2M_score', '_scvi_batch', '_scvi_labels', 'C_scANVI']]
SCC0120_1_pbmc_annotated